In [1]:
import numpy as np
import pandas as pd
import pickle
import time
import datetime
import sqlite3

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [3]:
df = pd.read_csv(path + "NYPD_Motor_Vehicle_Collisions.csv", parse_dates=[['DATE', 'TIME']], infer_datetime_format=True)

Load df and drop all except unique key and date time / date

In [4]:
df.set_index(['UNIQUE KEY'], inplace=True)

In [5]:
df['day'] = df.DATE_TIME.dt.date
df.day = df.day.astype('<M8[ns]')

# Read in sunset sunrise data

In [6]:
daylight = pd.read_csv(path + "sunrise_sunset.csv")

In [7]:
daylight.set_index(['Year', 'Day'], inplace=True)
daylight = pd.DataFrame(daylight.stack(level = 0)).reset_index()

Clean data to be in a better format

In [8]:
daylight['month'] = [i.strip('()').split(', ')[0] for i in daylight.level_2]
daylight['rise_set'] = [i.strip('()').split(', ')[1] for i in daylight.level_2]
daylight.rename(columns={0: 'value'}, inplace= True)

In [9]:
final = daylight[daylight.rise_set == 'Rise'].reset_index()

In [10]:
final['Set Time'] = daylight[daylight.rise_set == 'Set']['value'].values
final['Rise Time'] = [int(x) for x in final.value]
final['Set Time'] = [int(x) for x in final['Set Time']]
final = final.loc[:,['Year', 'Day', 'month', 'Rise Time', 'Set Time']]

Change month string to number

In [11]:
month_mapper = dict(January=1, February=2, March=3, April=4, May=5, June=6, July=7, August=8, September=9,\
                   October=10, November=11, December=12)
final['month'] = final['month'].map(month_mapper)

Remove invalid dates

In [12]:
final['day'] = pd.to_datetime(final.loc[:,['Year', 'Day', 'month']], errors='coerce')
final['day_str'] = final['day'].astype(str)
final = final[final['day_str'] != 'NaT']

In [13]:
rise = final.copy()
rise['Hour'] = np.floor(rise['Rise Time'] / 100).astype(int)
rise['Minute'] = rise['Rise Time'] % 100
rise = rise.loc[:, ['Year', 'Day', 'month', 'Hour', 'Minute']]

sunset = final.copy()
sunset['Hour'] = np.floor(sunset['Set Time'] / 100).astype(int)
sunset['Minute'] = sunset['Set Time'] % 100
sunset = sunset.loc[:, ['Year', 'Day', 'month', 'Hour', 'Minute']]

In [14]:
final['rise_time'] = pd.to_datetime(rise)
final['set_time'] = pd.to_datetime(sunset)
final = final.loc[:, ['day', 'rise_time', 'set_time']]

In [15]:
final.set_index('day', inplace=True)

In [16]:
df = df.loc[:, ['DATE_TIME', 'day']]

In [17]:
df = df.join(final, on='day', how='left')

In [23]:
sun = []

for idx, row in df.iterrows():
    sun.append((row.DATE_TIME > row.rise_time) & (row.DATE_TIME <= row.set_time))

df['sun_out'] = sun

In [25]:
df = df.loc[:,['rise_time', 'set_time', 'sun_out']]

In [27]:
with open('daylight_joined.pkl', 'wb') as output:
    pickle.dump(df, output, pickle.HIGHEST_PROTOCOL)